In [19]:
import sys
sys.path.append("../src/") 
import os

import model_manipulation as mm
import cobra
import cplex
import libsbml
import pandas as pd
import copy


In [20]:
#Read 2-cell model
model = cobra.io.read_sbml_model("../model/ios2164_2cell.xml")
#Estimate inf
inf = 1e6


In [21]:
#Solver
model.solver = 'cplex'

In [22]:
#Initialize medium so as to induce Photoautotrophic conditions (i.e. no Sucrose import)
mm.read_medium_csv('../misc/photo_medium.csv', model)

#Test natin kung magwowork sa est. infinite
#Yes setting bounds to 1000 works
model_media = model.medium
model.medium

{'EX_co2(e)': 1000000,
 'EX_h2o(e)': 1000000,
 'EX_no3(e)': 1000000,
 'EX_o2(e)': 1000000,
 'EX_pi(e)': 1000000,
 'EX_so4(e)': 1000000,
 'EX_so3(e)': 1000000,
 'EX_h2s(e)': 1000000,
 'EX_nh4(e)': 1000000,
 'EX_fe2(e)': 1000000,
 'EX_fe3(e)': 1000000,
 'EX_mg2(e)': 1000000,
 'EX_photonVis(e)': 1000000}

In [23]:
# #Add constraints to model

# #BS photon flux must be the same/less than M flux (Adapted from B&B, 2019)
# photon_import = model.reactions.get_by_id("EX_photonVis(e)")

BS_photon_import = model.reactions.PRISM_blue_LED_BS
M_photon_import = model.reactions.PRISM_blue_LED_M

photon_flux = model.problem.Constraint(M_photon_import.flux_expression 
                                       - BS_photon_import.flux_expression,
                                       lb = 0, ub = 1000)
model.add_cons_vars(photon_flux)


co2tex_m = model.reactions.CO2tex_M
co2tex_bs = model.reactions.CO2tex_BS

#CO2 Uptake must be constrained to A (Net assimilation rate) which is around 29 +- 1.2 umol CO2 m-2 s-1 for wild type rice
co2_import_cons = model.problem.Constraint(co2tex_m.flux_expression 
                                           + co2tex_bs.flux_expression, lb = 0, ub = 29)
model.add_cons_vars(co2_import_cons)

#CO2 BS intake must be limited owing to its position in the leaf tissue

model.add_cons_vars(model.problem.Constraint(co2tex_bs.flux_expression, lb = 0, ub = 2))
# #Note: CO2 uptake is not restricted in BS cells but will be constrained to be  equal/less than M cell

# co2_flux_cons = model.problem.Constraint(BS_co2_import.flux_expression - M_co2_import.flux_expression, lb = 0, ub = inf)
# model.add_cons_vars(co2_flux_cons)

# #Oks na

# #CO2 intake needs to be mostly in the M cells. Not sure of the exact value:
# Values from Von Caemmerer (2000) in mixed C3-C4 photosynthetic plants is 1 umol CO2 m-2 s-1 per 20 CO2 m-2 s-1. 
# #try checking Danila et al. (2016) on ratio of surface area of M cell to BS cell
# #I'll assume that the ratio is at 9.5:0.5 in the meantime, meaning that there is minimal gas exchange into the BS cell
#co2_ratio_cons = mm.set_fix_flux_ratio({co2tex_m.id:10, co2tex_bs.id:1},model)


# #Automatic na palang nalalagay siya as constraint

# #First is the BS and M Cell constraints

In [ ]:
#model.remove_cons_vars(co2_ratio_cons)

In [ ]:
# print(co2_flux_cons)

In [ ]:
# model.remove_cons_vars(co2_import_cons)
# model.remove_cons_vars(co2_flux_cons)

In [25]:
#What if I restrict yung import ng O2?

#Let's try restricting O2 to the value from Lakshmanan et al. (2016) = 3.312 mmol O2 m-2 s-1
#Actually this value is also reflected in Von Caemmerer's "Modelling C3 metabolism" (pg. 85)
o2tex_M = model.reactions.O2tex_M
o2tex_BS = model.reactions.O2tex_BS
o2_import_cons = model.problem.Constraint((o2tex_M.flux_expression + o2tex_BS.flux_expression)-, lb = 0, ub = 0)
model.add_cons_vars(o2_import_cons)

In [34]:
model.remove_cons_vars(o2_import_cons)

In [26]:
# #Add enzyme constraints

# #PEPC
wt_pepc = 2.4 #umol m-2 s-1
wt_mdh = 11.18 #umol m-2 s-1
wt_nadp_me = 0.14 #umol m-2 s-1
wt_ppdk = 0.31 #umol m-2 s-1
wt_CA = 15.74 #umol m-2  s-1

#PEPC constraint (Reaction id: PPCc)
pepc_BS = model.reactions.PPCc_BS
pepc_M = model.reactions.PPCc_M

wt_pepc_cons = model.problem.Constraint(pepc_BS.flux_expression 
                                        + pepc_M.flux_expression, 
                                        lb = 0, ub = wt_pepc)
model.add_cons_vars(wt_pepc_cons)

#PPDK constraints (Reaction id: PPDKS) (note that this is found in the chloroplast?) 
#Not detected via immunolocalization but enzyme activity is detected
ppdks_BS = model.reactions.PPDKs_BS
ppdks_M = model.reactions.PPDKs_M

wt_ppdks_cons = model.problem.Constraint(ppdks_BS.flux_expression 
                                         + ppdks_M.flux_expression, 
                                         lb = 0, ub = wt_ppdk)
model.add_cons_vars(wt_ppdks_cons)

#Malate Dehydrogenase 
mdh2c_M = model.reactions.MDH2c_M
mdh2c_BS = model.reactions.MDH2c_BS
mdh2s_M = model.reactions.MDH2s_M
mdh2s_BS = model.reactions.MDH2s_BS

wt_mdh_cons = model.problem.Constraint(mdh2c_M.flux_expression 
                                       + mdh2c_BS.flux_expression 
                                       + mdh2s_M.flux_expression 
                                       + mdh2s_BS.flux_expression, 
                                       lb= 0, ub=wt_mdh)
model.add_cons_vars(wt_mdh_cons)


#NADP-ME (Since no signal is detected in WT, no locational constraints are imposed)
nadp_me_M = model.reactions.MDH2s_M
nadp_me_BS = model.reactions.MDH2s_BS

wt_nadpme_cons = model.problem.Constraint(nadp_me_M.flux_expression
                                         + nadp_me_BS.flux_expression,
                                         lb= 0, ub=wt_nadp_me)
model.add_cons_vars(wt_nadpme_cons)
#Weird, it causes the activity of RBCL to significantly vary from the actual values

#Rbcl constaints

In [27]:
#Modifications to medium

model_media['EX_photonVis(e)'] = 1500 #PPFD used in Ermakova et al for high light

#Set CO2 bounds to ~29 m-2 s-1
model_media['EX_co2(e)'] = inf
model_media['EX_o2(e)'] = inf

#Test if heterotrophic conditions fix Biomass
#model_media['EX_sucr(e)_M'] = inf

#Nope, still doesn't produce biomass.
#How about Protons?
model_media['EX_h(e)'] = 0
#What if I constrained nitrogen uptake?
model_media['EX_no3(e)'] = inf
model_media['EX_nh4(e)'] = inf
model.medium = model_media


#Try nga natin may gas exchange in both.
# model.reactions.get_by_id('EX_co2(e)_M').upper_bounds = inf
# model.reactions.get_by_id('EX_co2(e)_BS').upper_bounds = inf

In [ ]:
#Let's try constraining yung input ng CO2 sa BS
# model.reactions.CO2tex_BS.bounds = (0,0)

In [ ]:
#Let's try nga changing yung model objective
# with model:
#     ex_sucr = model.reactions.get_by_id('EX_sucr(e)')
#     model.reactions.get_by_id('EX_sucr(e)').bounds = (0, 10000000)
#     #Set model objective to max sucrose
#     ex_sucr.objective_coefficient = 1.0
#     print(model.objective)


In [35]:
sample_fluxes = cobra.flux_analysis.pfba(model)
sample_fluxes

,fluxes,reduced_costs
Coleoptile_Biomass_M,0.000000,6.128352e+04
Straw_Biomass_M,4.822064,-2.000000e+00
ACCOAC_OSAc_M,0.000000,2.000000e+00
ACCOAC_OSAr_M,0.486845,-2.000000e+00
ACPC_OSs_M,0.000000,-2.000000e+00
...,...,...
dhap_pd,0.000000,6.821210e-13
g3p_pd,0.000000,6.821210e-13
3pg_pd,0.000000,1.932676e-12
glyclt_pd,0.000000,-2.000000e+00


In [39]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
h2s_e,EX_h2s(e),0.0883,0,0.00%
nh4_e,EX_nh4(e),5.193,0,0.00%
o2_e,EX_o2(e),263.9,0,0.00%
photonVis_e,EX_photonVis(e),1500,0,0.00%
pi_e,EX_pi(e),1.672,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
coa_c1,DM_COA(c)_BS,-3.31E-05,21,40.48%
coa_c0,DM_COA(c)_M,-4.868E-05,21,59.52%
photonDrain_u1,DM_photonDrain(u)_BS,-168.7,0,0.00%
photonDrain_u0,DM_photonDrain(u)_M,-314.6,0,0.00%


In [37]:
model.metabolites.co2_e.summary()

Percent,Flux,Reaction,Definition
Percent,Flux,Reaction,Definition


In [38]:
#This is cool, the M cell is similar to the WT rate of 83.59+-5.63 :)
print('rbcl M cell: ', sample_fluxes['RBPCs_M'], 'rbcl BS cell: ',sample_fluxes['RBPCs_BS'])
print('rbcl M cell (photorespiration)', sample_fluxes['RBPOs_M'], 'rbcl BS cell (PR)', sample_fluxes['RBPOs_BS'])
print('Biomass M: ', sample_fluxes['Straw_Biomass_M'], 'Biomass BS', sample_fluxes['Straw_Biomass_BS'])
print('co2 consumption M', sample_fluxes[co2tex_m.id], 'co2 consumption BS', sample_fluxes[co2tex_bs.id])


rbcl M cell:  262.803899600098 rbcl BS cell:  178.65984398077836
rbcl M cell (photorespiration) 0.0 rbcl BS cell (PR) 0.0
Biomass M:  4.8220643648023 Biomass BS 3.278902820023999
co2 consumption M 0.0 co2 consumption BS 0.0


In [ ]:
model.metabolites.o2_c1.summary()

In [ ]:
for items in model.exchanges:
    print(items.id, sample_fluxes[items.id])

In [ ]:
#Questions:
#Should I restrict Nutrient flow to the Bundle Sheath Cell only or should I allow flux to both the M and BS cell?
#Apparently kasi dito only the Bundle sheath cells produce any biomass
#This is to ensure that only CO2 and Light flux are the only limiting constraints to the system.